In [1]:
import formulas

In [37]:
# Parse the formula into an AST
ast = formulas.Parser().ast('=B2+SUM(A1:A3)')


In [47]:
pipe = ast[1].compile()

In [59]:
pipe.inputs

OrderedDict([('A1:A3', <Ranges>(A1:A3)), ('B2', <Ranges>(B2))])

In [57]:
pipe(1,2)

Array(3.0, dtype=object)

Get inputs delta using the above method.

Evaluate by passing in the inputs into the pipe() function


SIMPLIFIES EVERYTHING SIGNIFICANTLY